# Проект 1. Кто хочет стать миллионером?

## Подключение модулей и чтение файла

In [2]:
import pandas as pd
from collections import Counter
from itertools import combinations

movies_imbd = pd.read_csv('movie_bd_v5.csv')  # чтение данных из CSV файла
display(movies_imbd.head(3))  # показ трех верхних строк
movies_imbd.info()  # информация по типу данных и их количестве
movies_imbd.describe()  # основные статистические характеристики данных

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


## Предобработка

In [4]:
# проверка на нулевые значения
count_nan = len(movies_imbd) - movies_imbd.count()
print(count_nan)

# корректировка формата даты (хотя в этом нет острой необходимости)
movies_imbd['release_date'] = movies_imbd['release_date'].str.replace('/', '.')

# добление колонки с подсчетом прибыли
movies_imbd['profit'] = movies_imbd['revenue'] - movies_imbd['budget']
answers = {}  # создание словаря для ответов


# функция для обозначения статуса_1 фильма
def profitable(x):
    if x > 0:
        return 'прибыльный'
    else:
        return 'убыточный'


# подсчет среднего бюджета фильмов
mean_budget = round(movies_imbd.budget.mean())


# функция для обозначения статуса_2 фильма
def h_budget(x):
    if x > mean_budget:
        return 'высокобюджетный'
    else:
        return 'малобюджетный'


# функция для вычленения месяца из даты
def month(s):
    return s[:s.find('.')]


# функция для вычленения в столбцах составляющих, где '|' является разделителем
def counter(data, x):  # вычленение составляющих
    fission = data[x].str.cat(sep='|')
    new_df = pd.Series(fission.split('|'))
    info = new_df.value_counts(ascending=False)
    return info


# создание новой DataFrame путем копирования исходной DF
# с целью сохранения последней в первоначальном виде (на всякий случай)
new_movies_imbd = movies_imbd.copy()

# добавление колонки с обозначением прибыльный фильм или убыточный
new_movies_imbd['status_1'] = new_movies_imbd.profit.apply(profitable)

# добавление колонки с обозначением высокобюджетный фильм или малобюджетный
new_movies_imbd['status_2'] = new_movies_imbd.budget.apply(h_budget)

# добавление колонки с обозначением месяца выхода фильма в прокат
new_movies_imbd['month'] = new_movies_imbd.release_date.apply(month)

# создание колонки с подсчетом количества символов в названии фильма
new_movies_imbd['title_symbol_length'] = movies_imbd.original_title.map(
    lambda x: len(x))

# создание колонки с подсчетом количества слов в описании фильма
new_movies_imbd['overview_words_length'] = movies_imbd.overview.map(
    lambda x: len(x.split(' ')))


# функция для определения времени года
def season(b):
    if b == '1' or b == '2' or b == '12':
        return 'зима'
    elif b == '3' or b == '4' or b == '5':
        return 'весна'
    elif b == '6' or b == '7' or b == '8':
        return 'лето'
    elif b == '9' or b == '10' or b == '11':
        return 'осень'


# добавление столбца с информацией о времени года
new_movies_imbd['season'] = new_movies_imbd.month.apply(season)

display(new_movies_imbd.head(3))  # показ 3 позиций новой DataFrame

imdb_id                 0
budget                  0
revenue                 0
original_title          0
cast                    0
director                0
tagline                 0
overview                0
runtime                 0
genres                  0
production_companies    0
release_date            0
vote_average            0
release_year            0
profit                  0
dtype: int64


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_date,vote_average,release_year,profit,status_1,status_2,month,title_symbol_length,overview_words_length,season
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,...,6.9.2015,6.5,2015,1363528810,прибыльный,высокобюджетный,6,14,26,лето
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,...,5.13.2015,7.1,2015,228436354,прибыльный,высокобюджетный,5,18,110,весна
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,...,3.18.2015,6.3,2015,185238201,прибыльный,высокобюджетный,3,9,22,весна


### 1. Фильм с самым большим бюджетом

ВАРИАНТ 1

In [5]:
# +
max_budget = movies_imbd.loc[movies_imbd['budget'] == movies_imbd['budget'].
                             max()]  # выбор строчки по условию
display(
    max_budget[['original_title', 'imdb_id', 'budget']]
)  # вывод на экран № п/п, названия фильма, его id и бюджет - для проверки на адекватность

,original_title,imdb_id,budget
723,Pirates of the Caribbean: On Stranger Tides,tt1298650,380000000


ВАРИАНТ 2

In [6]:
# +
budget_max = movies_imbd['budget'].idxmax()
print(budget_max, movies_imbd.iloc[budget_max, 3], '(',
      movies_imbd.iloc[budget_max, 0],
      ')')  # вывод на экран № п/п, названия фильма и его id в скобках

723 Pirates of the Caribbean: On Stranger Tides ( tt1298650 )


ЗАПИСЬ ОТВЕТА

In [7]:
answers.setdefault(
    '1', '723 Pirates of the Caribbean: On Stranger Tides (tt1298650)')
print(answers['1'])

723 Pirates of the Caribbean: On Stranger Tides (tt1298650)


### 2. Самый длинный фильм (в минутах)

ВАРИАНТ 1

In [8]:
# +
max_runtime = movies_imbd.loc[movies_imbd['runtime'] == movies_imbd['runtime'].
                              max()]  # выбор строчки по условию
display(
    max_runtime[['original_title', 'imdb_id', 'runtime']]
)  # вывод на экран № п/п, названия фильма, его id и длительность - для проверки на адекватность

,original_title,imdb_id,runtime
1157,Gods and Generals,tt0279111,214


ВАРИАНТ 2

In [9]:
# +
top_runtime = movies_imbd['runtime'].idxmax()
print(top_runtime, movies_imbd.iloc[top_runtime, 3], '(',
      movies_imbd.iloc[top_runtime, 0],
      ')')  # вывод на экран № п/п, названия фильма и его id в скобках

1157 Gods and Generals ( tt0279111 )


ЗАПИСЬ ОТВЕТА

In [10]:
answers.setdefault('2', '1157 Gods and Generals (tt0279111)')
print(answers['2'])

1157 Gods and Generals (tt0279111)


### 3. Самый короткий фильм (в минутах)

ВАРИАНТ 1

In [11]:
# +
min_runtime = movies_imbd.loc[movies_imbd['runtime'] == movies_imbd['runtime'].
                              min()]  # выбор строчки по условию
display(min_runtime[[
    'original_title', 'imdb_id', 'runtime'
]])  # вывод на экран № п/п, названия фильма, его id и длительность

,original_title,imdb_id,runtime
768,Winnie the Pooh,tt1449283,63


ВАРИАНТ 2

In [12]:
# +
least_runtime = movies_imbd['runtime'].idxmin()
print(least_runtime, movies_imbd.iloc[least_runtime, 3], '(',
      movies_imbd.iloc[least_runtime, 0], ')')

768 Winnie the Pooh ( tt1449283 )


ЗАПИСЬ ОТВЕТА

In [13]:
answers.setdefault('3', '768 Winnie the Pooh (tt1449283)')
print(answers['3'])

768 Winnie the Pooh (tt1449283)


### 4. Средняя длительность фильмов

ВАРИАНТ 1

In [14]:
print(round(movies_imbd.runtime.mean(), 2), '- средняя длительность фильмов')

109.66 - средняя длительность фильмов


ЗАПИСЬ ОТВЕТА

In [15]:
answers.setdefault('4', '109.66 - средняя длительность фильмов')
print(answers['4'])

109.66 - средняя длительность фильмов


### 5. Mедианное значение длительности фильмов

ВАРИАНТ 1

In [16]:
# +
print(round(movies_imbd.runtime.median(), 2),
      '- медианное значение длительности фильмов')

107.0 - медианное значение длительности фильмов


 ЗАПИСЬ ОТВЕТА

In [17]:
answers.setdefault('5', '107.0 - медианное значение длительности фильмов')
print(answers['5'])

107.0 - медианное значение длительности фильмов


### 6. Самый прибыльный фильм

ВАРИАНТ 1

In [18]:
# +
max_profit = movies_imbd.loc[movies_imbd['profit'] == movies_imbd['profit'].
                             max()]  # выбор строчки по условию
display(max_profit[[
    'original_title', 'imdb_id', 'profit'
]])  # вывод на экран № п/п, названия фильма, его id и прибыль

,original_title,imdb_id,profit
239,Avatar,tt0499549,2544505847


ВАРИАНТ 2

In [19]:
# +
profit_max = movies_imbd['profit'].idxmax()
print(profit_max, movies_imbd.iloc[profit_max, 3], '(',
      movies_imbd.iloc[profit_max, 0], ')')

239 Avatar ( tt0499549 )


ЗАПИСЬ ОТВЕТА

In [20]:
answers.setdefault('6', '239 Avatar (tt0499549)')
print(answers['6'])

239 Avatar (tt0499549)


### 7. Самый убыточный фильм

ВАРИАНТ 1

In [21]:
# +
min_profit = movies_imbd.loc[movies_imbd['profit'] == movies_imbd['profit'].
                             min()]  # выбор строчки по условию
display(min_profit[[
    'original_title', 'imdb_id', 'profit'
]])  # вывод на экран № п/п, названия фильма, его id и убыток

,original_title,imdb_id,profit
1245,The Lone Ranger,tt1210819,-165710090


ВАРИАНТ 2

In [22]:
# +
profit_min = movies_imbd['profit'].idxmin()
print(profit_min, movies_imbd.iloc[profit_min, 3], '(',
      movies_imbd.iloc[profit_min, 0], ')')

1245 The Lone Ranger ( tt1210819 )


ЗАПИСЬ ОТВЕТА

In [23]:
answers.setdefault('7', '1245 The Lone Ranger (tt1210819)')
print(answers['7'])

1245 The Lone Ranger (tt1210819)


### 8. Количество фильмов, объем сборов которых оказался выше бюджета

ВАРИАНТ 1

In [24]:
# +
count = 0
for i in movies_imbd['profit']:
    if i > 0:
        count += 1
print(count, 'фильмов, объем сборов которых оказался выше бюджета')

1478 фильмов, объем сборов которых оказался выше бюджета


ВАРИАНТ 2

In [25]:
# +
prifit_movies = new_movies_imbd[new_movies_imbd.status_1.str.contains(
    'прибыльный', na=False)]
prifit_movies.status_1.count()

1478

ЗАПИСЬ ОТВЕТА

In [26]:
answers.setdefault('8',
                   '1478 фильмов, объем сборов которых оказался выше бюджета')
print(answers['8'])

1478 фильмов, объем сборов которых оказался выше бюджета


### 9. Cамый кассовый фильм в 2008 году

In [27]:
# +
revenue_max = movies_imbd[movies_imbd.release_year == 2008].revenue.idxmax()
print(revenue_max, movies_imbd.iloc[revenue_max, 3], '(',
      movies_imbd.iloc[revenue_max, 0], ')')

599 The Dark Knight ( tt0468569 )


ЗАПИСЬ ОТВЕТА

In [28]:
answers.setdefault('9', '599 The Dark Knight (tt0468569)')
print(answers['9'])

599 The Dark Knight (tt0468569)


### 10. Самый убыточный фильм за период с 2012 по 2014 г.

ВАРИАНТ 1

In [29]:
# +
period = movies_imbd.query('2012<release_year<2014').sort_values(by='profit')
display(period[['original_title', 'imdb_id', 'profit']].head(1))

,original_title,imdb_id,profit
1245,The Lone Ranger,tt1210819,-165710090


ВАРИАНТ 2

In [30]:
# +
period = movies_imbd.query('2012<release_year<2014')
least_profit = period.profit.idxmin()
print(least_profit, movies_imbd.iloc[least_profit, 3], '(',
      movies_imbd.iloc[least_profit, 0], ')')

1245 The Lone Ranger ( tt1210819 )


ЗАПИСЬ ОТВЕТА

In [31]:
answers.setdefault('10', '1245 The Lone Ranger (tt1210819)')
print(answers['10'])

1245 The Lone Ranger (tt1210819)


### 11. Какого жанра фильмов больше всего

In [32]:
# +
count_movie_max = Counter()
for x in movies_imbd['genres'].str.split('|'):
    count_movie_max += Counter(x)
count_movie_max.most_common(1)

# Counter - вид словаря,
# позволяющий считать количество неизменяемых объектов

# most_common([n]) -
# возвращает n наиболее часто встречающихся элементов,
# в порядке убывания встречаемости

[('Drama', 782)]

In [33]:
answers.setdefault('11', 'Drama')
print(answers['11'])

Drama


### 12. Фильмы какого жанра чаще всего становятся прибыльными?

ВАРИАНТ 1

In [34]:
# + ?
count_movie = Counter()
for x in new_movies_imbd.loc[new_movies_imbd['status_1'].map(
        lambda x: True
        if 'прибыльный' in x else False)]['genres'].str.split('|'):
    count_movie += Counter(x)
count_movie.most_common(1)

[('Drama', 560)]

ВАРИАНТ 2

In [35]:
# - (ошибочное решение)
new_movies_imbd
prifit_movies = new_movies_imbd[new_movies_imbd.status_1.str.contains(
    'прибыльный', na=False)]
prifit_movies
display(prifit_movies['genres'].describe(include=['object']))

count       1478
unique       548
top       Comedy
freq         121
Name: genres, dtype: object

ЗАПИСЬ ОТВЕТА

In [36]:
answers.setdefault('12', 'Drama')
print(answers['12'])

Drama


### 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [37]:
# +
top_director = movies_imbd.groupby(
    ['director'])['revenue'].sum().sort_values(ascending=False)
display(top_director.index[0])

'Peter Jackson'

In [38]:
answers.setdefault('13', 'Peter Jackson')
print(answers['13'])

Peter Jackson


### 14. Какой режисер снял больше всего фильмов в стиле Action?

In [39]:
# +
count_movie = Counter()
for x in movies_imbd.loc[movies_imbd['genres'].map(
        lambda x: True
        if 'Action' in x else False)]['director'].str.split('|'):
    count_movie += Counter(x)
count_movie.most_common(1)

[('Robert Rodriguez', 9)]

In [40]:
answers.setdefault('14', 'Robert Rodriguez')
print(answers['14'])

Robert Rodriguez


### 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году?

In [41]:
# +
actor = movies_imbd[movies_imbd['release_year'] == 2012]

count = Counter()
for i in range(0, len(actor)):
    for k in actor.iloc[i]['cast'].split('|'):
        count[k] += actor.iloc[i]['revenue']
count.most_common(1)

[('Chris Hemsworth', 2027450773)]

In [42]:
answers.setdefault('15', 'Chris Hemsworth')
print(answers['15'])

Chris Hemsworth


### 16. Актер, который снялся в большем количестве высокобюджетных фильмов

In [43]:
# +
# используем функцию counter, написанную самостоятельно в предобработке
sum_movie = counter(
    movies_imbd[movies_imbd['budget'] > movies_imbd['budget'].mean()], 'cast')
display(sum_movie.index[0])

'Matt Damon'

In [44]:
answers.setdefault('16', 'Matt Damon ')
print(answers['16'])

Matt Damon 


### 17. В фильмах какого жанра больше всего снимался Nicolas Cage?

In [45]:
# +
count_genres = Counter()
for x in movies_imbd.loc[movies_imbd['cast'].map(
        lambda x: True
        if 'Nicolas Cage' in x else False)]['genres'].str.split('|'):
    count_genres += Counter(x)
display(count_genres.most_common(1))

[('Action', 17)]

In [67]:
answers.setdefault('17', 'Action')
print(answers['17'])

Action


### 18. Самый убыточный фильм от Paramount Pictures

In [47]:
# +
pp = movies_imbd[movies_imbd.production_companies.str.contains(
    'Paramount Pictures', na=False)]
min_profit_pp = pp.loc[pp['profit'] == pp['profit'].min()]
display(min_profit_pp[['original_title', 'imdb_id', 'profit']])

,original_title,imdb_id,profit
925,K-19: The Widowmaker,tt0267626,-64831034


In [46]:
answers.setdefault('18', 'K-19: The Widowmaker')
print(answers['18'])

K-19: The Widowmaker


### 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [48]:
# +
top_year = movies_imbd.groupby(
    ['release_year'])['revenue'].sum().sort_values(ascending=False)
display(top_year.index[0])

2015

In [49]:
answers.setdefault('19',
                   '2015 - самый успешный год по суммарным кассовым сборам')
print(answers['19'])

2015 - самый успешный год по суммарным кассовым сборам


### 20. Какой самый прибыльный год для студии Warner Bros?

In [50]:
# +
wb = movies_imbd[movies_imbd.production_companies.str.contains('Warner Bros',
                                                               na=False)]
pd.DataFrame(wb.groupby(by='release_year').profit.sum()).sort_values(
    'profit', ascending=False).head(1)

,profit
release_year,
2014,2295464519


In [51]:
answers.setdefault('20', '2014 - самый прибыльный год для студии Warner Bros')
print(answers['20'])

2014 - самый прибыльный год для студии Warner Bros


### 21. В каком месяце за все годы суммарно вышло больше всего фильмов?¶

In [52]:
# +
month = new_movies_imbd.month
month.describe(
)  # смотрим на top - самое часто встречаемое значение
# (число от 1 до 12 - значение месяца)

count     1889
unique      12
top          9
freq       227
Name: month, dtype: object

In [53]:
answers.setdefault('21', 'сентябрь')
print(answers['21'])

сентябрь


### 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [54]:
# +
month = new_movies_imbd.month
amount = 0
for i in month:
    if (i == '6') or (i == '7') or (i == '8'):
        amount += 1
amount

450

In [55]:
answers.setdefault('22', '450 фильмов суммарно вышло за лето')
print(answers['22'])

450 фильмов суммарно вышло за лето


### 23. Для какого режиссера зима – самое продуктивное время года?

In [56]:
# +
count_genres = Counter()
for x in new_movies_imbd.loc[new_movies_imbd['season'].map(
        lambda x: True if 'зима' in x else False)]['director'].str.split('|'):
    count_genres += Counter(x)
display(count_genres.most_common(1))

[('Peter Jackson', 7)]

In [57]:
answers.setdefault('23', 'Peter Jackson')
print(answers['23'])

Peter Jackson


### 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [58]:
# +
sum_sym = counter(new_movies_imbd, 'production_companies')
for sym in sum_sym.index:
    sum_sym[sym] = new_movies_imbd['title_symbol_length'][
        new_movies_imbd['production_companies'].map(
            lambda x: True if sym in x else False)].mean()
sum_sym = pd.DataFrame(sum_sym).sort_values(0, ascending=False)
sum_sym.head(1)

,0
Four By Two Productions,83


In [59]:
answers.setdefault('24', 'Four By Two Productions')
print(answers['24'])

Four By Two Productions


### 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [60]:
# +
sum_words = counter(new_movies_imbd, 'production_companies')
for word in sum_words.index:
    sum_words[word] = new_movies_imbd['overview_words_length'][
        new_movies_imbd['production_companies'].map(
            lambda x: True if word in x else False)].mean()
sum_words = pd.DataFrame(sum_words).sort_values(0, ascending=False)
sum_words.head(1)

,0
Midnight Picture Show,175


In [61]:
answers.setdefault('25', 'Midnight Picture Show')
print(answers['25'])

Midnight Picture Show


### 26. Какие фильмы входят в 1 процент лучших по рейтингу? 


In [62]:
# +
top_1 = movies_imbd[movies_imbd['vote_average'] > movies_imbd.quantile(
    0.99, numeric_only=True)['vote_average']]
display(top_1[['original_title', 'imdb_id', 'vote_average']])

,original_title,imdb_id,vote_average
9,Inside Out,tt2096673,8.0
34,Room,tt3170832,8.0
118,Interstellar,tt0816692,8.0
119,Guardians of the Galaxy,tt2015381,7.9
125,The Imitation Game,tt2084970,8.0
128,Gone Girl,tt2267998,7.9
138,The Grand Budapest Hotel,tt2278388,7.9
370,Inception,tt1375666,7.9
599,The Dark Knight,tt0468569,8.1
872,The Pianist,tt0253474,7.9


In [63]:
answers.setdefault('26', 'Inside Out, The Dark Knight, 12 Years a Slave')
print(answers['26'])

Inside Out, The Dark Knight, 12 Years a Slave


### 27. Какие актеры чаще всего снимаются в одном фильме вместе?

In [64]:
# +
actor_data = movies_imbd.cast.str.split('|').tolist()
list_actors = []
for i in actor_data:
    for j in combinations(i, 2):
        list_actors.append(' '.join(j))
list_actors = pd.DataFrame(list_actors)
list_actors.columns = ['actor_together']
list_actors.actor_together.value_counts().head(10)

Daniel Radcliffe Rupert Grint       8
Daniel Radcliffe Emma Watson        8
Rupert Grint Emma Watson            7
Johnny Depp Helena Bonham Carter    6
Ben Stiller Owen Wilson             6
Hugh Jackman Ian McKellen           5
Kristen Stewart Taylor Lautner      5
Robert Pattinson Taylor Lautner     5
Adam Sandler Kevin James            5
Kristen Stewart Robert Pattinson    5
Name: actor_together, dtype: int64

In [65]:
answers.setdefault('27', 'Daniel Radcliffe & Rupert Grint')
print(answers['27'])

Daniel Radcliffe & Rupert Grint


### Просмотр всех ответов

In [68]:
for key in answers.keys():
    print(key, ':', answers[key])

1 : 723 Pirates of the Caribbean: On Stranger Tides (tt1298650)
2 : 1157 Gods and Generals (tt0279111)
3 : 768 Winnie the Pooh (tt1449283)
4 : 109.66 - средняя длительность фильмов
5 : 107.0 - медианное значение длительности фильмов
6 : 239 Avatar (tt0499549)
7 : 1245 The Lone Ranger (tt1210819)
8 : 1478 фильмов, объем сборов которых оказался выше бюджета
9 : 599 The Dark Knight (tt0468569)
10 : 1245 The Lone Ranger (tt1210819)
11 : Drama
12 : Drama
13 : Peter Jackson
14 : Robert Rodriguez
15 : Chris Hemsworth
16 : Matt Damon 
18 : K-19: The Widowmaker
19 : 2015 - самый успешный год по суммарным кассовым сборам
20 : 2014 - самый прибыльный год для студии Warner Bros
21 : сентябрь
22 : 450 фильмов суммарно вышло за лето
23 : Peter Jackson
24 : Four By Two Productions
25 : Midnight Picture Show
26 : Inside Out, The Dark Knight, 12 Years a Slave
27 : Daniel Radcliffe & Rupert Grint
17 : Action
